# Machine Learning Part 3: Neural Network

This file is load the data and see the result

In [1]:
import tensorflow as tf
import numpy as np

import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 2

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.95 # was 0.8


In [2]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,64])), # was 32, 64
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,64,64])),
               #                                  64 features
               'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,32])),
               #                                  64 features
               'W_conv4':tf.Variable(tf.random_normal([3,3,3,32,32])),
               'W_fc':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc2':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc3':tf.Variable(tf.random_normal([1024,1024])),# was 54080, 1024
               'out':tf.Variable(tf.random_normal([1024, n_classes]))} # was 1024

    biases = {'b_conv1':tf.Variable(tf.random_normal([64])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_conv3':tf.Variable(tf.random_normal([32])),
                'b_conv4':tf.Variable(tf.random_normal([32])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
              'b_fc2':tf.Variable(tf.random_normal([1024])),
              'b_fc3':tf.Variable(tf.random_normal([1024])),# was 16
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.leaky_relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.leaky_relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    
    conv3 = tf.nn.leaky_relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = maxpool3d(conv3)
    
    conv4 = tf.nn.leaky_relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = maxpool3d(conv4)
    print(conv4.get_shape)
    fc = tf.reshape(conv4,[-1, 1024])# was 54080
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    fc = tf.nn.leaky_relu(tf.matmul(fc, weights['W_fc3'])+biases['b_fc3'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output


In [3]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [4]:
#much_data = np.load('muchdata-50-50-20.npy')



#train_data = much_data[:-30]
#validation_data = much_data[-30:]
#train_data = much_data
# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
much_data = np.load('muchdata-50-50-20-b.npy')
train_data = [[item[0],item[1]] for item in much_data]
#test_data = train_data
test_data = np.load('testdata-50-50-20-b.npy')
test_data = [[item[0],item[1],item[2]] for item in test_data]

P_list = []
proba_list = []
proba_list_2 = []

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y ))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
    
    hm_epochs = 300
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            #correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            #for i in xrange(10):
            #    val_data = chunkIt(validation_data,10)[i]
            #    print('Accuracy:',accuracy.eval({x:[i[0] for i in val_data], y:[i[1] for i in val_data]}))
            #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
            #print accuracy
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            predictions = tf.argmax(prediction,1)
            proba = tf.cast(prediction, 'float')
            proba_2 = tf.nn.softmax(logits=prediction)
                        
        print('Done. Finishing accuracy:')
        for j in xrange(30):
            te_data = chunkIt(test_data,30)[j]
            print('Accuracy:',accuracy.eval({x:[i[0] for i in te_data], y:[i[1] for i in te_data]}))
            P_list.append(predictions.eval({x: [i[0] for i in te_data]}))
            proba_list.append(proba.eval({x: [i[0] for i in te_data]}))
            proba_list_2.append((proba_2.eval({x: [i[0] for i in te_data]})))
            
        #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('fitment percent:',successful_runs/total_runs)

# Run this locally:
train_neural_network(x)

<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_3:0' shape=(?, 4, 4, 2, 32) dtype=float32>>
Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 1, 'completed out of', 300, 'loss:', 50691316736000.0)
('Epoch', 2, 'completed out of', 300, 'loss:', 24508234366976.0)
('Epoch', 3, 'completed out of', 300, 'loss:', 21608032239616.0)
('Epoch', 4, 'completed out of', 300, 'loss:', 15403039899648.0)
('Epoch', 5, 'completed out of', 300, 'loss:', 12196432113664.0)
('Epoch', 6, 'completed out of', 300, 'loss:', 14337509916672.0)
('Epoch', 7, 'completed out of', 300, 'loss:', 8728640618496.0)
('Epoch', 8, 'completed out of', 300, 'loss:', 7503711027200.0)
('Epoch', 9, 'completed out of', 300, 'loss:', 5190380208128.0)
('Epoch', 10, 'completed out of', 300, 'loss:', 5897634791424.0)
('Epoch', 11, 'completed out of', 300, 'loss:', 4640704512000.0)
('Epoch', 12, 'completed out of', 300, 'loss:', 3818908499968.0)
('Epoch', 13, 'completed out of', 300, 'loss:', 3

('Epoch', 131, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 132, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 133, 'completed out of', 300, 'loss:', 544670412800.0)
('Epoch', 134, 'completed out of', 300, 'loss:', 829159146496.0)
('Epoch', 135, 'completed out of', 300, 'loss:', 148432109568.0)
('Epoch', 136, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 137, 'completed out of', 300, 'loss:', 114197561344.0)
('Epoch', 138, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 139, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 140, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 141, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 142, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 143, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 144, 'completed out of', 300, 'loss:', 189186068480.0)
('Epoch', 145, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 146, 'completed out of', 300, 'loss:', 61744799744.0)
('Epoch', 147, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 148, 'c

('Epoch', 279, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 280, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 281, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 282, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 283, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 284, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 285, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 286, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 287, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 288, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 289, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 290, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 291, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 292, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 293, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 294, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 295, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 296, 'completed out of', 300, 'loss:', 0.0)
('Epoch', 297, 'completed ou

In [5]:
feature_test = [np.sum(img[0]) for img in test_data]

In [6]:
labels = pd.read_csv('../../cap2input/stage2_solution.csv', index_col=0)
labels.cancer.value_counts()

0    353
1    153
Name: cancer, dtype: int64

In [7]:
proba_list

[array([[  6.63804576e+11,   1.17082071e+11],
        [  4.71574053e+11,   6.41644757e+11],
        [  4.58964304e+11,   5.98536225e+11],
        [  6.63089316e+11,  -2.51885060e+11],
        [  1.44291765e+12,  -3.72188185e+11],
        [  1.01595644e+12,  -5.69734988e+11],
        [  1.22486994e+12,   1.00557259e+11],
        [  5.84418263e+11,   2.89482408e+11],
        [  6.91209372e+11,   3.42503588e+11],
        [  6.43098214e+11,   1.37787163e+11],
        [  1.35993937e+12,  -6.19240358e+11],
        [  5.96713603e+11,  -1.85512460e+11],
        [  5.10073242e+11,   1.89392814e+11],
        [  2.34799366e+11,   1.94673590e+11],
        [  8.84894466e+11,   3.57804671e+11],
        [  8.24169791e+11,  -3.12264589e+11]], dtype=float32),
 array([[  6.49198895e+11,  -1.59216075e+11],
        [  1.06004192e+12,  -3.78532463e+11],
        [  1.60563449e+12,   6.83707335e+11],
        [  7.94772505e+11,   1.40717523e+11],
        [  8.98725642e+11,  -5.81259362e+11],
        [  8.7587

In [8]:
P_list[0]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [9]:
ori_list = [item for sublist in P_list for item in sublist]
print ori_list

true_list = [item[1][1] for item in test_data]
print true_list

patient_list = [item[2] for item in test_data]

[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
ori_list_proba = [item for sublist in proba_list for item in sublist]

ori_list_proba_true = [[-sublist[1],-sublist[0]] if (sublist[0]+sublist[1]<0) else sublist for sublist in ori_list_proba]


proba_list_true = [sublist/sum(sublist) for sublist in ori_list_proba_true]

proba_list_0 = [sublist[0] for sublist in proba_list_true]
proba_list_1 = [0.2 if sublist[1]<0.2 else 0.8 if sublist[1]>0.8 else sublist[1] for sublist in proba_list_true]

#ori_list_proba = [np.exp(item) for sublist in proba_list for item in sublist]

print proba_list_1

[0.2, 0.57638687, 0.5659914, 0.2, 0.2, 0.2, 0.2, 0.33125323, 0.33133337, 0.2, 0.2, 0.2, 0.27076769, 0.45328489, 0.28792542, 0.2, 0.2, 0.2, 0.298648, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.63357198, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2012337, 0.2, 0.5886631, 0.2, 0.2, 0.56213272, 0.21899056, 0.2, 0.2, 0.20347793, 0.2, 0.8, 0.2, 0.2, 0.43154702, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.22058082, 0.2, 0.51275885, 0.58903033, 0.2, 0.2, 0.2, 0.2, 0.30019197, 0.62596101, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25087714, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.5470475, 0.42158034, 0.2, 0.2, 0.40352568, 0.25604728, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.66695184, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.8, 0.39817449, 0.45441529, 0.2, 0.63279331, 0.2708033, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.40004656, 0.6125657, 0.2, 0.4864352, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.5271371, 0.2, 0

In [11]:
#ori_list_proba_2 = [item for sublist in proba_list_2 for item in sublist]
#print ori_list_proba_2

In [12]:
labels['proba'] = proba_list_1
labels['predictions'] = ori_list
labels['true'] = true_list
labels['feature'] = feature_test
labels['patient'] = patient_list


In [13]:
pred = labels[['patient','proba']].reset_index(drop=True)
pred.columns = ['id','cancer']

In [14]:
#pred[pred == 0] = 0.4
#pred[pred == 1] = 0.6

In [15]:
pred

,id,cancer
0,7a632e7b8309e4b57a5964762a9e594a,0.200000
1,d662c2884bf723e2b0b2401ccab08887,0.576387
2,edde23221a8c4038f9a10daa3088ee94,0.565991
3,91541739b4a106e5481a6156c5b3cfbb,0.200000
4,d82af7a99fb96b2df37b4a0db270795f,0.200000
5,2b76a32e5c12e4312cb029721a24b190,0.200000
6,307a08ece9a33b772c903f5afbf9a799,0.200000
7,5d669e1f9abdff1a282b924c828a9b0a,0.331253
8,ad690e1d0df849ee7983f69ab43e785c,0.331333
9,c6c77fd425da262d6eeed53cff38cd91,0.200000


In [16]:
pred.to_csv('new_sixth_sub.csv', index=False)